In [3]:
from PExp import *

In [4]:
exp1 = PExp(6,population=['b','b','a','c','a','b','b','c'],prior_prob=[1,1],n_trials=50000)
exp2 = PExp(6,population={'b':4,'c':2,'a':2},prior_prob=[1,1])
exp3 = PExp(population=[('a','R'),('a','R'),('b','B'),('b','R')])
exp4 = PExp(6,population=[('b',),('b',),('a',),('c',),('a',),('b',),('b',),('c',)],prior_prob=[1,1])
exp5 = PExp(prior_prob=[1,2,1])
exp6 = PExp(prior_prob=[1,2,1],alphabet=['a','b','c'])
exp7 = PExp(prior_prob=[1,2,1],alphabet=[('a','0'),('b','1'),('c','2')])
exp8 = PExp(alphabet=['a','b','c'])
exp9 = PExp(4)
#exp1.info()
#exp2.info()
#exp3.info()
#exp4.info()
#exp5.info()
#exp6.info()
#exp7.info()
#exp8.info()
#exp9.info()
#exp1.info()
#exp1.make_exp()
#exp1.info()

exp1 = PExp(6)
exp2 = PExp(alphabet=['T','C'])
exp1.info()
exp2.info()
exp3 = exp1*exp2
exp3.info()
exp3.make_exp(10000)
exp3.info()

----------------------------------------------------------------------------------------------------
Population: None
Alphabet: [('1',), ('2',), ('3',), ('4',), ('5',), ('6',)]
Prior_prob: [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
N_outcomes: 6
N_trials: 6000
Post_prob: [ 0.  0.  0.  0.  0.  0.]
Exp_dict:
{('1',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('2',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('3',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('4',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('5',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('6',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666}}

----------------------------------------------------------------------------------------------------
Population: None
Alphabet: [('T',), ('C',)]
Prior_prob: [ 0.5  0.5]
N_outcomes: 2
N_trials: 2000
Post_prob: [ 0.  0.]
Exp_dict:
{('C',): {'post_prob': 0.0, 'prior_prob': 0.5},

In [ ]:
exp_carte = PExp(alphabet=['A','2','3','4','5','6','7','8','9','J','Q','K'])
exp_semi = PExp(alphabet=['C','Q','F','P'])
exp_cartefrancesi = exp_carte * exp_semi
exp_cartefrancesi.info()

In [ ]:
class Event():
    def __init__(self,abc,exp):
        if type(exp) != PExp:
            raise TypeError('Exp must be a member of PExp class')
            
        if type(abc) != list:
            raise TypeError('Elements must be a list')
        
        fle = [True]*len(exp.alphabet)
        
        self.exp = exp
        
        tabc = []
        # Cicla su tutti gli elementi dell'evento
        for el in abc:
            if '*' not in el:
                tabc.append(el)
            else:
                for k in range(len(el)):
                    if el[k] != '*':
                        for j in range(len(exp.alphabet)):
                            if el[k] != exp.alphabet[j][k]:
                                fle[j] = False
                            
                for k in range(len(fle)):
                    if fle[k] == True:
                        tabc.append(exp.alphabet[k])
        
        self.abc = []
        for el in tabc:
            if el in exp.alphabet:
                self.abc.append(el)
                        
        self.prob = 0.0
        for el in self.abc:
            if el in exp.exp_dict:
                self.prob += exp.exp_dict[el]['prior_prob']
        
        
    def get_prob(self):
        return self.prob
        
        
        
    def __neg__(self):
        abctemp = []
        for k in self.exp.alphabet:
            if k not in self.abc:
                abctemp.append(k)
        return Event(abctemp,self.exp)
    
    def __add__(self,right):
        if type(right) != Event:
            raise TypeError('All the operands must be Event objects')
        abctemp = list(set(self.abc + right.abc)) # Lists concatenation
        abctemp.sort()
        return Event(abctemp,self.exp)
    
    def __mul__(self,right):
        if type(right) != Event:
            raise TypeError('All the operands must be Event objects')
        abctemp = []
        for k in self.abc:
            if k in right.abc:
                abctemp.append(k)
        return Event(abctemp,self.exp)
    
    def __or__(self,right):
        if type(right) != Event:
            raise TypeError('All the operands must be Event objects')
        exptemp = PExp(alphabet=right.abc)
        evtemp = Event(self.abc,exptemp)
        return Event(evtemp.abc,exptemp)
    
    def __repr__(self):
        return(str(self.abc)+': '+str(self.prob))
    def __str__(self):
        return(str(self.abc)+': '+str(self.prob))

In [ ]:
e1 = Event([('K','C'),('6','F')],exp_cartefrancesi)
print(e1.get_prob())
print(e1)
Fraction(e1.get_prob()).limit_denominator(100)

In [ ]:
e2 = Event([('*','C')],exp_cartefrancesi)
print(e2.get_prob())
print(e2)
Fraction(e2.get_prob()).limit_denominator(100)

In [ ]:
e3 = Event([('J','*')],exp_cartefrancesi)
print(e3.get_prob())
print(e3)
Fraction(e3.get_prob()).limit_denominator(100)

In [ ]:
print((-e3).get_prob() + e3.get_prob())
print(e3 + (-e3))

In [ ]:
e3|e2

In [ ]:
Fraction(0.0833333333333).limit_denominator(100)

In [ ]:
exptemp = PExp(alphabet=e2.abc)
exptemp.info()
evtemp = Event(e3.abc,exptemp)
print(e3.abc)
evtemp.abc
#evtemp.prob

In [2]:
from PExp import *